This notebooks help to prepare dataset for training flow matching model.

We convert images to VAE latents and saves them locally

In [ ]:
#folder with images dataset
images_folder='/home/vlad/Documents/TreesDataset'

# where to save VAE latents
save_latents_dir = './latents_tree'

# image size for VAE
image_size=256

# generate also horizontal flip version for each image
add_flip = True

In [ ]:
import os
import torchvision.transforms as T
from kemsekov_torch.utils import PadToMultiple
import tqdm
from vae import encode
from clip_emb import *

clip = CLIPEmbedder()

device='cuda'

tr = T.Compose([
    T.ToTensor(),
    T.Resize(image_size,T.InterpolationMode.NEAREST),
    T.RandomHorizontalFlip(),
])
flip = T.RandomHorizontalFlip()

pad = PadToMultiple(64)

image_formats=['.png','.jpg','.jpeg','.webp']
def save(clip, device, pad, save_folder, i, f, im):
    latents = encode(pad(im),device).half()
    emb = clip.image_to_embedding(f).half()
    torch.save(
            {
                "im_path":f,
                'vae_latents':latents,
                'clip_emb':emb,
            },
            os.path.join(save_folder,str(i)+".pt")
        )

for dir_path,subf,files in os.walk(images_folder):
    files = [os.path.join(dir_path,f) for f in files if [f.endswith(v) for v in image_formats]]
    class_ = os.path.split(dir_path)[-1]
    if len(files)==0: continue
    save_folder = os.path.join(save_latents_dir,class_)
    os.makedirs(save_folder,exist_ok=True)
    files_r = files
    data = tqdm.tqdm(enumerate(files_r),total=len(files_r),desc=class_)
    for i,f in data:
        im=tr(Image.open(f).convert("RGB"))
        save(clip, device, pad, save_folder, i, f, im)
        if add_flip:
            save(clip, device, pad, save_folder, str(i)+"_flip", f, flip(im))
            